<a href="https://colab.research.google.com/github/Juliana-Ribeiro-Lopes/PLN/blob/main/Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2025-Q3]**
Prof. Alexandre Donizeti Alves

###**PROJETO PRÁTICO COP30**





Este notebook implementa um pipeline de Processamento de Linguagem Natural (PLN) utilizando o framework LangChain integrado ao modelo de linguagem Gemini (Google Generative AI). O objetivo é analisar como a mídia internacional tem discutido a COP30, que ocorrerá em Belém (PA), a partir de dados coletados em tempo real via NewsAPI.

Foram aplicadas três técnicas principais de PLN:

1. Busca Semântica - foi usada para encontrar as notícias mais relevantes sobre a COP30 dentro do corpus retornado pela API. Para isso, utilizamos embeddings + FAISS para recuperar documentos mais relevantes.

2. Extração de tópicos - utilizado a estração das informações importantes de cada notícia retornada pela busca semântica, usando um LLM com esquema Pydantic para extrair fatos estruturados.


3. Sumarização de textos - utilizada para condensar automaticamente as notícias coletadas, permitindo identificar rapidamente os temas centrais abordados pela imprensa.


Por fim, todas as informações processadas são integradas em uma análise meta-sintética, que gera insights estratégicos sobre narrativas, riscos, oportunidades e lacunas de comunicação relacionadas à COP30. O projeto demonstra como ferramentas modernas de PLN podem apoiar tomadas de decisão e estudos de políticas climáticas com base em dados textuais.

# Equipe

---

**Integrante 1:**


```
Ana Luísa Costa Nunes | 11202420185
```

**Integrante 2**


```
Juliana Ribeiro Lopes | 21075612
```





# Grande modelo de linguagem (Large Language Model - LLM)



---



LLM: Gemini 2.5-flash

Link: https://docs.cloud.google.com/vertex-ai/generative-ai/docs/models/gemini/2-5-flash

**Link para a documentação oficial**: https://ai.google.dev/gemini-api/docs/models?hl=pt-br


#API


---


API: News API

Site oficial: https://newsapi.org/

Link para a documentação oficial: https://newsapi.org/docs

**DESCRIÇÃO**
---

Implementar um `notebook` no `Google Colab` que faça uso do framework **`LangChain`** (obrigatório) e de um **LLM** aplicando, no mínimo, DUAS técnicas de PLN. As técnicas podem ser aplicada em qualquer córpus obtido a partir de uma **API** ou a partir de uma página Web.

O **LLM** e a **API** selecionados devem ser informados na seguinte planilha:

> https://docs.google.com/spreadsheets/d/1iIUZcwnywO7RuF6VEJ8Rx9NDT1cwteyvsnkhYr0NWtU/edit?usp=sharing

>
As seguintes técnicas de PLN podem ser usadas:

*   Correção Gramatical
*   Classificação de Textos
*   Análise de Sentimentos
*   Detecção de Emoções
*   Extração de Palavras-chave
*   Tradução de Textos
*   Sumarização de Textos
*   Similaridade de Textos
*   Reconhecimento de Entidades Nomeadas
*   Sistemas de Perguntas e Respostas
>

**IMPORTANTE:** É obrigatório usar o e-mail da UFABC.

#Implementação


---



##LangChain

In [1]:
!pip install -U --force-reinstall langchain langchain-google-genai google-generativeai newsapi-python tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 1.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usu

In [1]:
#@title Versão do LangChain

import langchain

print(langchain.__version__)

1.1.0


In [2]:
#@title Integração com o pacote da Gemini
# The installation is now handled in the cell above to ensure dependency compatibility.
!pip install -U langchain-google-genai

  Using cached langchain_google_genai-3.2.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached google_ai_generativelanguage-0.9.0-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.9 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.4.0
    Uninstalling google-ai-generativelanguage-0.4.0:
      Successfully uninstalled google-ai-generativelanguage-0.4.0
  Attempting uninstall: langchain-google-genai
    Found existing installation: langchain-google-genai 0.0.1
    Uninstalling langchain-google-genai-0.0.1:
      Successfully uninstalled langchain-google-genai-0.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depe

## Chaves

In [3]:
from getpass import getpass

GOOGLE_API_KEY = getpass()

··········


In [4]:
from getpass import getpass

NEWS_API_KEY = getpass()

··········


## Pesquisa sobre a COP 30

In [9]:
import requests

url  = ('https://newsapi.org/v2/everything?'
       'q=cop30&'
       'from=2025-11-10&' #inicio da COP30
       'to=2025-11-21&' #fim da COP30
       'language=pt&' #ar de en es fr he it nl no pt ru sv ud zh
       'sortBy=popularity&' # relevancy popularity publishedAt
       'apiKey=' + NEWS_API_KEY)

res = requests.get(url)

In [10]:
noticias_json = res.json()

# Verificação de erro
if noticias_json.get("status") == "error":
    print("Erro na API:", noticias_json.get("message"))
else:
    noticias = noticias_json.get("articles", [])
    print("Total de notícias:", len(noticias))
    print(noticias)

Total de notícias: 78
[{'source': {'id': None, 'name': 'Uol.com.br'}, 'author': 'Isabela Oliveira', 'title': 'IA vai salvar o clima ou turbinar a crise? Debate esquenta na COP30 em Belém', 'description': 'Na COP30, órgãos apresentam iniciativa de "IA do bem" contra crise climática; defensores do meio ambiente discordam\nThe post IA vai salvar o clima ou turbinar a crise? Debate esquenta na COP30 em Belém appeared first on Giz Brasil.', 'url': 'https://gizbr.uol.com.br/ia-vai-salvar-o-clima-ou-turbinar-a-crise-debate-esquenta-na-cop30-em-belem/', 'urlToImage': 'https://gizbr.uol.com.br/wp-content/blogs.dir/8/files/2023/05/belem-para.jpg', 'publishedAt': '2025-11-19T11:31:20Z', 'content': 'Em meio à COP30, em Belém (PA), defensores da inteligência artificial (IA) estão argumentando que a tecnologia pode ajudar a reduzir a resolver a crise do clima. Mesmo que ela seja associada a um gra… [+2756 chars]'}, {'source': {'id': None, 'name': 'Uol.com.br'}, 'author': 'Agência Fapesp', 'title': '

Criar documentos LangChain


*   Cada artigo vira um documento do LangChain, com page_content e metadata.




In [11]:
from langchain_core.documents import Document

documentos = []

for artigo in noticias:
    doc = Document(
        page_content=artigo.get("content", "") or artigo.get("description", ""),
        metadata={
            "titulo": artigo.get("title"),
            "autor": artigo.get("author"),
            "url": artigo.get("url"),
            "data": artigo.get("publishedAt")
        }
    )
    documentos.append(doc)


***`Splitting`***

Quebra os textos em chunks menores. Cada chunk é enviado para geração de embeddings e depois indexado no FAISS.
E overlap de 200 caratcteres que preserva o contexto

In [14]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(documentos)

print(len(all_splits))

78


Criando Embeddings

In [17]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key = GOOGLE_API_KEY)

In [18]:
#@title Instalando FAISS
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 40.5 MB/s eta 0:00:00


Indexar os documentos para busca semântica utilizando FAISS

In [19]:
from langchain_community.vectorstores import FAISS

faiss_index = FAISS.from_documents(all_splits, embeddings)

Busca Semântica

In [20]:
query = "Principais decisões da COP30"
docs_relevantes = faiss_index.similarity_search(query, k=5) # devolde os 5 mais similares
print(docs_relevantes[0])

page_content='Siga o Olhar Digital no Google Discover
Nesta quarta-feira (19), foi divulgado o rascunho da carta final da 30ª Conferência das Nações Unidas sobre Mudanças Climáticas (COP30). Nele, há várias recom… [+5815 chars]' metadata={'titulo': 'COP30: proposta contra aquecimento aparece em rascunho de carta final', 'autor': 'Rodrigo Mozelli', 'url': 'https://olhardigital.com.br/2025/11/19/ciencia-e-espaco/cop30-proposta-contra-aquecimento-aparece-em-rascunho-de-carta-final/', 'data': '2025-11-20T00:51:52Z', 'start_index': 0}


In [21]:
for i, doc in enumerate(docs_relevantes, start=1):
    print(f"\n=== Documento {i} ===")
    print(doc.page_content)
    print("URL:", doc.metadata.get("url"))


=== Documento 1 ===
Siga o Olhar Digital no Google Discover
Nesta quarta-feira (19), foi divulgado o rascunho da carta final da 30ª Conferência das Nações Unidas sobre Mudanças Climáticas (COP30). Nele, há várias recom… [+5815 chars]
URL: https://olhardigital.com.br/2025/11/19/ciencia-e-espaco/cop30-proposta-contra-aquecimento-aparece-em-rascunho-de-carta-final/

=== Documento 2 ===
Siga o Olhar Digital no Google Discover
Esta sexta-feira (21) marca o último dia da 30ª Conferência das Nações Unidas sobre Mudanças Climáticas (COP30), realizada em Belém, no Pará. No entanto, as n… [+4615 chars]
URL: https://olhardigital.com.br/2025/11/21/ciencia-e-espaco/cop30-chega-ao-ultimo-dia-que-avancos-ainda-sao-possiveis/

=== Documento 3 ===
Os governadores de Estados brasileiros enviaram uma carta ao presidente da 30ª Conferência das Nações Unidas sobre as Mudanças Climáticas (COP30), André Corrêa do Lago, pedindo que as resoluções do e… [+2680 chars]
URL: https://valor.globo.com/brasil/cop30-a

Extrai um esquema

In [22]:
from typing import Optional, List

from pydantic import BaseModel, Field

class Artigo(BaseModel):
    topico: str = Field(default=None, description="Assunto principal da notícia")
    decisao: str = Field(default=None, description="Qual a decisão ou ação tomada?")
    paises: list[str] = Field(default=None, description="Os países envolvidos ou citados")
    orgao: str = Field(default=None, description="Qual o órgão responsável?")
    categoria: str = Field(default=None, description="Qual a categoria da notícia: ambiental, econômica ou diplomática?")
    fonte: str = Field(default=None, description="A fonte da notícia")


class ExtracaoDados(BaseModel):
      """Informações principais extraídas sobre as decisões da COP 30."""

      desenvolvimentos: List[Artigo]

      def __str__(self):
              return self.model_dump_json(indent=2, ensure_ascii=False)


In [23]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

# Defina um prompt personalizado para fornecer instruções e qualquer contexto adicional.
# 1) Você pode adicionar exemplos ao modelo de prompt para melhorar a qualidade da extração.
# 2) Introduza parâmetros adicionais para levar o contexto em consideração (por exemplo, incluir metadados
#    sobre o documento do qual o texto foi extraído.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um analista especializado em jornalismo, relações internacionais, políticas ambientais e diplomacia global."
            "Sua função é extrair fatos relevantes de notícias jornalísticas, documentos oficiais e comunicados diplomáticos."
            "Extraia apenas fatos importantes. Não extraia nada se não houver informações relevantes no texto."
            "Preencha os campos do esquema exatamente como definidos na estrutura Pydantic.",
        ),
        ("human", "{texto}"),
    ]
)

In [24]:
entrada = ""

for i, d in enumerate(docs_relevantes, start=1):
    entrada += f"\n=== ARTIGO {i} ===\n"
    entrada += d.page_content + "\n"


Cria um Extrator

In [25]:
from langchain_google_genai import ChatGoogleGenerativeAI

modelo = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, google_api_key = GOOGLE_API_KEY)

extrator = prompt | modelo.with_structured_output( schema = ExtracaoDados, include_raw = False)

In [26]:
resultado = extrator.invoke({"texto": entrada})
print(str(resultado))

{
  "desenvolvimentos": [
    {
      "topico": "Rascunho da carta final da COP30",
      "decisao": "Divulgação do rascunho da carta final da COP30",
      "paises": [],
      "orgao": "30ª Conferência das Nações Unidas sobre Mudanças Climáticas (COP30)",
      "categoria": "Ambiental",
      "fonte": "Olhar Digital"
    },
    {
      "topico": "Último dia da COP30 e negociações",
      "decisao": "Marca o último dia da 30ª Conferência das Nações Unidas sobre Mudanças Climáticas (COP30)",
      "paises": [
        "Brasil"
      ],
      "orgao": "30ª Conferência das Nações Unidas sobre Mudanças Climáticas (COP30)",
      "categoria": "Ambiental",
      "fonte": "Olhar Digital"
    },
    {
      "topico": "Carta de governadores brasileiros à presidência da COP30",
      "decisao": "Governadores de Estados brasileiros enviaram uma carta pedindo resoluções à COP30",
      "paises": [
        "Brasil"
      ],
      "orgao": "Governadores de Estados brasileiros, 30ª Conferência das Naç

In [27]:

for i, art in enumerate(resultado.desenvolvimentos, start=1):
    print(f"\n=== ARTIGO {i} ===")
    print(f"Tópico: {art.topico}")
    print(f"Decisão: {art.decisao}")
    print(f"Países: {', '.join(art.paises) if art.paises else 'Nenhum'}")
    print(f"Órgão: {art.orgao}")
    print(f"Categoria: {art.categoria}")
    print(f"Fonte: {art.fonte}")
    print("==========================")



=== ARTIGO 1 ===
Tópico: Rascunho da carta final da COP30
Decisão: Divulgação do rascunho da carta final da COP30
Países: Nenhum
Órgão: 30ª Conferência das Nações Unidas sobre Mudanças Climáticas (COP30)
Categoria: Ambiental
Fonte: Olhar Digital

=== ARTIGO 2 ===
Tópico: Último dia da COP30 e negociações
Decisão: Marca o último dia da 30ª Conferência das Nações Unidas sobre Mudanças Climáticas (COP30)
Países: Brasil
Órgão: 30ª Conferência das Nações Unidas sobre Mudanças Climáticas (COP30)
Categoria: Ambiental
Fonte: Olhar Digital

=== ARTIGO 3 ===
Tópico: Carta de governadores brasileiros à presidência da COP30
Decisão: Governadores de Estados brasileiros enviaram uma carta pedindo resoluções à COP30
Países: Brasil
Órgão: Governadores de Estados brasileiros, 30ª Conferência das Nações Unidas sobre as Mudanças Climáticas (COP30)
Categoria: Ambiental
Fonte: Não especificada

=== ARTIGO 4 ===
Tópico: Crise do clima em destaque na COP30
Decisão: Não especificada
Países: Brasil
Órgão: COP